In [2]:
import datetime
import pandas as pd
import numpy as np

In [4]:
# 공휴일 데이터 불러오기
df_holiday = pd.read_csv("D:/humanda/project/1st_miniproject/Mook/hd_21_24.csv")

In [ ]:
# 데이터 확인
print(df_holiday.head(10))
print(df_holiday.info())

In [ ]:
# 공휴일 데이터의 일시를 날짜 데이터로 변경
df_holiday['일시'] = pd.to_datetime(df_holiday['일시'])

In [ ]:
# 날짜 데이터 생성(2022-01-01 ~ 2024-10-31 )
date_range = pd.date_range(start='2022-01-01', end='2024-10-31', freq='D')

In [ ]:
# 데이터프레임으로 저장
df_date = pd.DataFrame(date_range, columns=['date'])

In [ ]:
# 날짜 데이터에 공휴일이거나 주말이면 1 아니면 0이라는 컬럼 추가
df_date['holiday_or_weekend'] = df_date['date'].apply(
    lambda x: 1 if (x.dayofweek >= 5 or x in df_holiday['일시'].values) else 0
)

In [ ]:
# 확인
print(df_date.head(10))
print(df_date.info())

In [ ]:
# db 연결
conn = pymysql.connect(host="localhost", port=3306, db="project", 
                       user="humanda", password="humanda")

In [ ]:
# 명령 도구 준비
cursor = conn.cursor()

In [ ]:
# 명령 실행 (테이블 생성)
sql = """create table dates
         (
            ID INT AUTO_INCREMENT PRIMARY KEY,         
            date date,
            holiday_or_weekend int
         )"""
cursor.execute(sql)

In [ ]:
# 명령 실행 (데이터 삽입)
sql2 = "insert into dates(date, holiday_or_weekend) values (%s, %s)"
for index, row in df_date.iterrows():
    cursor.execute(sql2, tuple(row))

In [ ]:
# 원본 테이블에 적용
conn.commit()

In [ ]:
# 명령 실행 (조회)
sql3 = "select * from dates"
cursor.execute(sql3)

In [ ]:
# 조회 결과 읽기
data = cursor.fetchall()

In [ ]:
# 표시
for row in data:
    print(row)

In [ ]:
# db에서 불러와 데이터프레임으로 저장
df_date = pd.read_sql_query(sql3, conn)

In [ ]:
df_date

In [ ]:
cursor.close()
conn.close()

In [ ]:
df_date.to_csv('df_date.csv', index=False)